<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [6]:
import pandas as pd

# Load the uploaded CSV
df = pd.read_csv('dataset_part_3.csv')

# Check the first few rows to confirm it loaded correctly
df.head()


,FlightNumber,PayloadMass,Orbit_ES-L1,Orbit_GEO,Orbit_GTO,Orbit_HEO,Orbit_ISS,Orbit_LEO,Orbit_MEO,Orbit_PO,...,Serial_B1048,Serial_B1049,Serial_B1050,Serial_B1051,Serial_B1054,Serial_B1056,Serial_B1058,Serial_B1059,Serial_B1060,Serial_B1062
0,1.0,6123.547647,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,525.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,677.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,500.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,3170.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
import pandas as pd

# Load dataset
df = pd.read_csv('dataset_part_1.csv')

# Take a quick look to confirm columns
df.head()


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857


In [10]:
# Get unique launch sites and their coordinates
launch_sites = df[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates()
launch_sites


,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
3,VAFB SLC 4E,34.632093,-120.610829
26,KSC LC 39A,28.608058,-80.603956


In [11]:
import folium

# Center map on average location of all launch sites
map_center = [launch_sites['Latitude'].mean(), launch_sites['Longitude'].mean()]

# Create a Folium map
site_map = folium.Map(location=map_center, zoom_start=5)

# Show the map
site_map


In [12]:
from folium.plugins import MarkerCluster

# Add launch sites as markers
marker_cluster = MarkerCluster().add_to(site_map)

for idx, row in launch_sites.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['LaunchSite'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

site_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [13]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [14]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [15]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [16]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [17]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


In [18]:
from folium.features import DivIcon

# Loop through each launch site
for idx, row in launch_sites.iterrows():
    coord = [row['Latitude'], row['Longitude']]
    
    # Add a circle
    folium.Circle(
        location=coord,
        radius=1000,           # radius in meters
        color='#000000',       # circle border color
        fill=True,
        fill_color='#blue',    # fill color
        fill_opacity=0.4
    ).add_child(folium.Popup(row['LaunchSite'])).add_to(site_map)
    
    # Add a marker with label
    folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['LaunchSite']
        )
    ).add_to(site_map)

# Display map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [25]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Load the original dataset
df = pd.read_csv('dataset_part_1.csv')  # Make sure this is the original CSV

# Check columns
print(df.columns)


Index(['FlightNumber', 'Date', 'BoosterVersion', 'PayloadMass', 'Orbit',
       'LaunchSite', 'Outcome', 'Flights', 'GridFins', 'Reused', 'Legs',
       'LandingPad', 'Block', 'ReusedCount', 'Serial', 'Longitude',
       'Latitude'],
      dtype='object')


In [27]:
import pandas as pd

df = pd.read_csv('dataset_part_1.csv')

# Create 'Class' column based on Outcome
df['Class'] = df['Outcome'].apply(lambda x: 1 if 'True' in str(x) else 0)

df[['Outcome', 'Class']].head(10)



,Outcome,Class
0,None None,0
1,None None,0
2,None None,0
3,False Ocean,0
4,None None,0
5,None None,0
6,True Ocean,1
7,True Ocean,1
8,None None,0
9,None None,0


In [28]:
import folium
from folium.plugins import MarkerCluster

# Base map
site_map = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)

# Marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add launches
for _, row in df.iterrows():
    coord = [row['Latitude'], row['Longitude']]
    color = 'green' if row['Class'] == 1 else 'red'
    
    folium.Circle(
        location=coord,
        radius=1000,
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=f"Launch Site: {row['LaunchSite']}\nOutcome: {row['Outcome']}"
    ).add_to(marker_cluster)

site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [29]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [30]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [32]:
# Create 'Class' column based on Outcome
spacex_df['Class'] = spacex_df['Outcome'].apply(lambda x: 1 if str(x).startswith('True') else 0)

# Now create marker_color column
spacex_df['marker_color'] = spacex_df['Class'].apply(lambda x: 'green' if x == 1 else 'red')

# Check results
spacex_df[['Outcome', 'Class', 'marker_color']].head(10)


<class 'KeyError'>: 'Outcome'

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [34]:
import numpy as np



In [35]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    r = 6371  # Earth radius in km
    return c * r


In [37]:
# Check all column names exactly as pandas sees them
print(spacex_df.columns.tolist())



['Launch Site', 'Lat', 'Long', 'class']


In [38]:
spacex_df.columns = spacex_df.columns.str.strip()


In [40]:
print(spacex_df.columns.tolist())



['Launch Site', 'Lat', 'Long', 'class']


In [41]:
spacex_df.columns = spacex_df.columns.str.strip()


In [42]:
print(spacex_df.columns)


Index(['Launch Site', 'Lat', 'Long', 'class'], dtype='object')


In [44]:
spacex_df.columns = spacex_df.columns.str.strip()


In [45]:
print(spacex_df.columns.tolist())


['Launch Site', 'Lat', 'Long', 'class']


In [47]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


In [48]:
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    # convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers
    return c * r


In [49]:
spacex_df['Distance_to_Equator_km'] = spacex_df.apply(
    lambda row: haversine(row['Lat'], row['Long'], 0, row['Long']),
    axis=1
)


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [50]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [51]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [52]:
import folium
from folium.plugins import MousePosition

# Example: create a map centered roughly in Florida/CCSFS
m = folium.Map(location=[28.5618571, -80.577366], zoom_start=5)

# Add MousePosition plugin
MousePosition(
    position='topright',
    separator=' | ',
    prefix='Coordinates:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 5);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 5);}",
).add_to(m)

m


In [53]:
coastline_lat = 28.5
coastline_lon = -80.6


In [54]:
spacex_df['Distance_to_Coastline_km'] = spacex_df.apply(
    lambda row: haversine(row['Lat'], row['Long'], coastline_lat, coastline_lon),
    axis=1
)


In [55]:
folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup='Closest Coastline Point',
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(m)

m


In [56]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [57]:
import folium

# Coordinates
launch_lat = 28.5618571   # Example: Kennedy Space Center
launch_lon = -80.577366

coastline_lat = 28.5       # Your manually picked coastline point
coastline_lon = -80.6

# Create the map
m = folium.Map(location=[launch_lat, launch_lon], zoom_start=6)


In [58]:
# Launch site marker
folium.Marker(
    location=[launch_lat, launch_lon],
    popup='Launch Site',
    icon=folium.Icon(color='red')
).add_to(m)

# Coastline marker
folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup='Coastline Point',
    icon=folium.Icon(color='blue')
).add_to(m)


In [59]:
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [coastline_lat, coastline_lon]],
    color='green',
    weight=3,
    opacity=0.7,
    tooltip='Distance Line'
).add_to(m)


In [60]:
m


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


In [61]:
# Example coordinates (replace with actual MousePosition coordinates)
city_lat, city_lon = 28.5721, -80.6480
railway_lat, railway_lon = 28.55, -80.6
highway_lat, highway_lon = 28.56, -80.59


In [62]:
# City marker
folium.Marker(
    location=[city_lat, city_lon],
    popup='Closest City',
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(m)

# Railway marker
folium.Marker(
    location=[railway_lat, railway_lon],
    popup='Closest Railway',
    icon=folium.Icon(color='purple', icon='train')
).add_to(m)

# Highway marker
folium.Marker(
    location=[highway_lat, highway_lon],
    popup='Closest Highway',
    icon=folium.Icon(color='orange', icon='road')
).add_to(m)


In [63]:
# Line to city
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [city_lat, city_lon]],
    color='blue',
    weight=2,
    opacity=0.6,
    tooltip='Distance to City'
).add_to(m)

# Line to railway
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [railway_lat, railway_lon]],
    color='purple',
    weight=2,
    opacity=0.6,
    tooltip='Distance to Railway'
).add_to(m)

# Line to highway
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [highway_lat, highway_lon]],
    color='orange',
    weight=2,
    opacity=0.6,
    tooltip='Distance to Highway'
).add_to(m)


In [64]:
m


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [65]:
# Launch site coordinates
launch_lat, launch_lon = 28.561857, -80.577366  # Example: CCSFS SLC 40

# Nearby features coordinates
city_coords = [28.5721, -80.6480]       # closest city
railway_coords = [28.55, -80.6]         # closest railway
highway_coords = [28.56, -80.59]        # closest highway


In [66]:
import folium

# Create map centered on launch site
m = folium.Map(location=[launch_lat, launch_lon], zoom_start=10)

# Launch site marker
folium.Marker(
    location=[launch_lat, launch_lon],
    popup='Launch Site',
    icon=folium.Icon(color='red', icon='rocket')
).add_to(m)

# Feature markers
features = {
    'City': city_coords,
    'Railway': railway_coords,
    'Highway': highway_coords
}

for name, coords in features.items():
    folium.Marker(
        location=coords,
        popup=f'Closest {name}',
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)


In [67]:
for name, coords in features.items():
    folium.PolyLine(
        locations=[[launch_lat, launch_lon], coords],
        color='green',
        weight=2,
        opacity=0.6,
        tooltip=f'Distance to {name}'
    ).add_to(m)


In [68]:
m


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
